In [28]:
import pandas as pd
import numpy as np
import scipy
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
import sklearn

In [29]:
feature_set = pd.read_csv('feature_matrix_w_position.csv')
feature_set = feature_set.drop(columns = ['Unnamed: 0'], axis = 1)

feature_set = feature_set.replace('H', 1)
feature_set = feature_set.replace('A', 0)

feature_set = feature_set.dropna()

feature_set.head(10)

,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,...,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points,primaryPositioncode,primaryPositionname,primaryPositiontype
885,8465009,371,2015,0,0.0,0.2,0.2,0.15,0.150000,0.0,...,0.0,0.0,0.0,0.0,0.05,0.050000,20.5,D,Defenseman,Defenseman
886,8465009,385,2015,0,1.0,0.4,0.3,0.20,0.190476,0.0,...,0.0,0.0,0.0,0.0,0.05,0.047619,20.8,D,Defenseman,Defenseman
887,8465009,396,2015,0,1.0,0.4,0.4,0.25,0.227273,1.0,...,0.0,0.0,0.0,0.0,0.05,0.045455,8.0,D,Defenseman,Defenseman
888,8465009,405,2015,1,0.0,0.4,0.3,0.25,0.217391,1.0,...,0.0,0.0,0.0,0.0,0.05,0.043478,4.3,D,Defenseman,Defenseman
889,8465009,417,2015,0,0.0,0.4,0.3,0.25,0.208333,0.0,...,0.0,0.0,0.0,0.0,0.00,0.041667,11.1,D,Defenseman,Defenseman
890,8465009,435,2015,1,0.0,0.4,0.3,0.25,0.200000,1.0,...,0.0,1.0,0.2,0.1,0.05,0.080000,2.8,D,Defenseman,Defenseman
891,8465009,448,2015,1,0.0,0.2,0.3,0.25,0.192308,0.0,...,0.0,0.0,0.2,0.1,0.05,0.076923,13.6,D,Defenseman,Defenseman
892,8465009,465,2015,1,0.0,0.0,0.2,0.20,0.185185,1.0,...,0.0,0.0,0.2,0.1,0.05,0.074074,1.5,D,Defenseman,Defenseman
893,8465009,477,2015,0,0.0,0.0,0.2,0.20,0.178571,0.0,...,0.0,0.0,0.2,0.1,0.05,0.071429,7.8,D,Defenseman,Defenseman
894,8465009,494,2015,1,0.0,0.0,0.2,0.20,0.172414,1.0,...,0.0,0.0,0.2,0.1,0.05,0.068966,13.3,D,Defenseman,Defenseman


Split into 11/1 days and days from before

In [30]:
feature_set_test_date = feature_set[(feature_set['season']==2019) & (feature_set['game_id'] >= 196) & (feature_set['game_id'] <= 203)]

feature_set_pre_test_date = feature_set[(feature_set['season']<2019) | (feature_set['season'] == 2019) & (feature_set['game_id'] < 196)]

In [31]:
X_pre = feature_set_pre_test_date.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_pre = feature_set_pre_test_date['dfs_points'].tolist()

feature_set_forward = feature_set_pre_test_date.loc[feature_set_pre_test_date['primaryPositioncode'].isin(['L','R','C'])]
X_forward = feature_set_forward.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_forward = feature_set_forward['dfs_points'].tolist()
                                      
feature_set_defense = feature_set_pre_test_date.loc[feature_set_pre_test_date['primaryPositioncode'].isin(['D'])]
X_defense = feature_set_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_defense = feature_set_defense['dfs_points'].tolist()

X_train_forward, X_test_forward, y_train_forward, y_test_forward = train_test_split(X_forward, y_forward, test_size=0.2, random_state=42)
X_train_defense, X_test_defense, y_train_defense, y_test_defense = train_test_split(X_defense, y_defense, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_pre, y_pre, test_size=0.2, random_state=42)


In [32]:
ridge = Ridge()
parameters = {'alpha':[1e-15, 1e-10,1e-5,1e-3,1e-2,1,5,10,20,100]}
ridge_regressor = RandomizedSearchCV(ridge, parameters, scoring = 'neg_mean_squared_error',cv=5)
ridge_regressor.fit(X_train_forward,y_train_forward)
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 100}
-46.65408281544557


## Forward Model

In [33]:
ridge = Ridge(alpha=100)
forward_model = ridge.fit(X_train_forward,y_train_forward)
forward_y = forward_model.predict(X_test_forward)
forward_y_train = forward_model.predict(X_train_forward)
score_forward_test = mean_squared_error(y_test_forward, forward_y)
score_forward_train = mean_squared_error(y_train_forward, forward_y_train)
print('Test: ' + str(score_forward_test))
print('Train: ' + str(score_forward_train))

Test: 44.81699540594442
Train: 46.54440293057694


## Defense Model

In [34]:
ridge = Ridge(alpha=100)
defense_model = ridge.fit(X_train_defense,y_train_defense)
defense_y = defense_model.predict(X_test_defense)
defense_y_train = defense_model.predict(X_train_defense)
score_defense_test = mean_squared_error(y_test_defense, defense_y)
score_defense_train = mean_squared_error(y_train_defense, defense_y_train)
print('Test: ' + str(score_defense_test))
print('Train: ' + str(score_defense_train))

Test: 29.739844859688745
Train: 29.432644831223815


## Total Model

In [75]:
ridge = Ridge(alpha=100)
total_model = ridge.fit(X_train,y_train)
y_test_pred = total_model.predict(X_test)
y_train_pred = total_model.predict(X_train)
score_total_test = mean_squared_error(y_test, y_test_pred)
score_total_train = mean_squared_error(y_train, y_train_pred)
#print('Test: ' + str(score_total_test))
#print('Train: ' + str(score_total_train))
#print(total_model.coef_)
import scipy.stats as ss
#print((total_model.coef_))
top_features = ss.rankdata(abs(total_model.coef_))==2
#print(abs(total_model.coef_))
#print(top_features)

cols = X_train.columns
#print(X_train*top_features)
print(cols[top_features==True])

Index(['es_toi_season'], dtype='object')


## Goalies

In [36]:
feature_set_G = pd.read_csv('goalie_feature_matrix.csv')
feature_set_G = feature_set_G.drop(columns = ['Unnamed: 0'], axis = 1)

feature_set_G = feature_set_G.replace('H', 1)
feature_set_G = feature_set_G.replace('A', 0)

feature_set_G = feature_set_G.dropna()

feature_set_G.head(10)

,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,...,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
24,8466141,454,2015,0,22.0,23.2,25.2,26.90,27.708333,2.0,...,0.2,0.5,0.45,0.625000,100.0,100.000000,87.5,93.750000,95.000000,25.6
25,8466141,479,2015,0,33.0,24.0,24.7,26.95,27.920000,1.0,...,0.6,0.5,0.50,0.680000,100.0,100.000000,100.0,93.750000,95.454545,26.8
30,8466141,572,2015,1,29.0,22.6,23.3,25.10,27.033333,1.0,...,0.4,0.5,0.45,0.633333,100.0,100.000000,100.0,91.666667,95.833333,41.0
31,8466141,589,2015,0,40.0,25.8,24.1,25.85,27.451613,0.0,...,0.6,0.6,0.50,0.645161,100.0,100.000000,100.0,92.857143,96.153846,24.2
33,8466141,632,2015,0,30.0,29.0,26.9,26.90,27.636364,2.0,...,1.0,0.8,0.60,0.666667,100.0,100.000000,100.0,93.750000,96.428571,10.4
36,8466141,707,2015,1,29.0,25.6,25.7,25.10,27.194444,2.0,...,0.6,0.6,0.50,0.638889,100.0,100.000000,100.0,100.000000,96.666667,28.9
38,8466141,736,2015,1,38.0,26.4,27.7,26.15,27.473684,2.0,...,0.4,0.7,0.50,0.631579,100.0,100.000000,100.0,100.000000,96.875000,17.9
41,8466141,795,2015,0,32.0,31.0,28.3,26.20,27.658537,3.0,...,0.4,0.5,0.55,0.609756,100.0,100.000000,100.0,100.000000,97.058824,13.0
44,8466141,877,2015,1,23.0,26.2,25.8,26.60,27.204545,5.0,...,0.6,0.5,0.60,0.613636,50.0,75.000000,87.5,93.750000,94.444444,22.8
45,8466141,909,2015,1,29.0,24.8,27.6,26.40,27.244444,1.0,...,0.8,0.6,0.55,0.622222,100.0,83.333333,90.0,93.750000,94.736842,14.4


In [37]:
feature_set_test_date_G = feature_set_G[(feature_set_G['season']==2019) & (feature_set_G['game_id'] >= 196) & (feature_set_G['game_id'] <= 203)]

feature_set_pre_test_date_G = feature_set_G[(feature_set_G['season']<2019) | (feature_set_G['season'] == 2019) & (feature_set_G['game_id'] < 196)]


In [38]:
X_G = feature_set_pre_test_date_G.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_G = feature_set_pre_test_date_G['dfs_points'].tolist()

X_train_G, X_test_G, y_train_G, y_test_G = train_test_split(X_G, y_G, test_size=0.2, random_state=42)

In [39]:
ridge = Ridge(alpha=100)
goalie_model = ridge.fit(X_train_G,y_train_G)
G_y_test = goalie_model.predict(X_test_G)
G_y_train = goalie_model.predict(X_train_G)
score_G_test = mean_squared_error(y_test_G, G_y_test)
score_G_train = mean_squared_error(y_train_G, G_y_train)
print('Test: ' + str(score_G_test))
print('Train: ' + str(score_G_train))

Test: 121.11682627215576
Train: 112.60676462507693


In [40]:
feature_set_forward = feature_set_test_date.loc[feature_set_test_date['primaryPositioncode'].isin(['L','R','C'])]
X_forward = feature_set_forward.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_forward = feature_set_forward['dfs_points'].tolist()
                                      
feature_set_defense = feature_set_test_date.loc[feature_set_test_date['primaryPositioncode'].isin(['D'])]
X_defense = feature_set_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season','primaryPositioncode','primaryPositionname','primaryPositiontype'], axis = 1)
y_defense = feature_set_defense['dfs_points'].tolist()

X_G = feature_set_test_date_G.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_G = feature_set_test_date_G['dfs_points'].tolist()


In [41]:
forward_prediction = (forward_model.predict(X_forward))
defense_prediction = (defense_model.predict(X_defense))
goalie_prediction = (goalie_model.predict(X_G))
forward_final = pd.DataFrame(feature_set_forward)
forward_final['Linear_DFS_Prediction'] = forward_prediction
defense_final = pd.DataFrame(feature_set_defense)
defense_final['Linear_DFS_Prediction'] = defense_prediction
goalie_final = pd.DataFrame(feature_set_test_date_G)
goalie_final['Linear_DFS_Prediction'] = goalie_prediction
final_predictions = pd.concat([forward_final,defense_final,goalie_final])

In [42]:
final_predictions=final_predictions[['player_id','Linear_DFS_Prediction']]

In [43]:
final_predictions

,player_id,Linear_DFS_Prediction
1732,8466138,6.109401
2124,8466139,6.814629
14045,8470047,6.013187
15032,8470144,4.188862
16665,8470257,6.367217
...,...,...
5647,8473541,11.521749
6492,8474593,13.498725
7099,8474651,13.269326
10218,8476412,13.629441


In [44]:
predicted_data = pd.read_csv('player_prices_w_pred.csv')
vijay_dat = pd.read_csv('final_predictions.csv')

L_5 = pd.read_csv('10_game_averages.csv')
L_5_G = pd.read_csv('10_game_averages_goalies.csv')
L_5 = pd.concat([L_5,L_5_G])

In [45]:
vijay_dat

,Unnamed: 0,player_id,NN_DFS_Prediction
0,1732,8466138,6.275066
1,2124,8466139,6.970334
2,14045,8470047,6.649446
3,15032,8470144,4.602513
4,16665,8470257,5.707118
...,...,...,...
256,5647,8473541,11.694061
257,6492,8474593,12.610475
258,7099,8474651,11.892563
259,10218,8476412,12.577880


In [46]:
L_5 = L_5[['fullName','player_id']]
L_5.head()

,fullName,player_id
0,Will Butcher,8477355
1,Oskar Lindblom,8478067
2,James van Riemsdyk,8474037
3,Travis Sanheim,8477948
4,Andy Greene,8472382


In [47]:
prediction_1 = final_predictions.merge(L_5, how='left',left_on='player_id',right_on = 'player_id')
prediction = predicted_data.merge(prediction_1, how='left',left_on='Name',right_on = 'fullName')
prediction = prediction.merge(vijay_dat,how='left',left_on='player_id',right_on='player_id')
prediction=prediction.drop(['fullName','player_id','Unnamed: 0_x','Unnamed: 0_y'], axis=1)
print(prediction)

    Position                     Name + ID               Name        ID  \
0          G  Jordan Binnington (13678944)  Jordan Binnington  13678944   
1          G   Philipp Grubauer (13678945)   Philipp Grubauer  13678945   
2         LW      Alex Ovechkin (13678140)      Alex Ovechkin  13678140   
3          G      Braden Holtby (13678946)      Braden Holtby  13678946   
4          G        Petr Mrazek (13678947)        Petr Mrazek  13678947   
..       ...                           ...                ...       ...   
275        D           Tim Heed (13678798)           Tim Heed  13678798   
276        D      Mario Ferraro (13678810)      Mario Ferraro  13678810   
277        C      Casey Cizikas (13678008)      Casey Cizikas  13678008   
278        C    Cedric Paquette (13678016)    Cedric Paquette  13678016   
279       LW      Ross Johnston (13678366)      Ross Johnston  13678366   

    Roster Position  Salary                      Game Info TeamAbbrev  \
0                 G    840

In [48]:
prediction = prediction.fillna(0)
print(prediction)
prediction.to_csv('player_prices_w_pred_Ben.csv')

    Position                     Name + ID               Name        ID  \
0          G  Jordan Binnington (13678944)  Jordan Binnington  13678944   
1          G   Philipp Grubauer (13678945)   Philipp Grubauer  13678945   
2         LW      Alex Ovechkin (13678140)      Alex Ovechkin  13678140   
3          G      Braden Holtby (13678946)      Braden Holtby  13678946   
4          G        Petr Mrazek (13678947)        Petr Mrazek  13678947   
..       ...                           ...                ...       ...   
275        D           Tim Heed (13678798)           Tim Heed  13678798   
276        D      Mario Ferraro (13678810)      Mario Ferraro  13678810   
277        C      Casey Cizikas (13678008)      Casey Cizikas  13678008   
278        C    Cedric Paquette (13678016)    Cedric Paquette  13678016   
279       LW      Ross Johnston (13678366)      Ross Johnston  13678366   

    Roster Position  Salary                      Game Info TeamAbbrev  \
0                 G    840

In [49]:
prediction[prediction['Position']=='G']

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,lgbm_projection,Linear_DFS_Prediction,NN_DFS_Prediction
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,14.82,13.629441,12.577880
1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,13.33,0.000000,0.000000
3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,13.90,13.269326,11.892563
4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,12.60,0.000000,0.000000
6,G,Andrei Vasilevskiy (13678949),Andrei Vasilevskiy,13678949,G,7900,TB@NYI 11/01/2019 07:00PM ET,TB,15.72,13.78,14.412698,12.497589
7,G,Martin Jones (13678948),Martin Jones,13678948,G,7800,WPG@SJ 11/01/2019 10:30PM ET,SJ,9.00,14.06,0.000000,0.000000
8,G,John Gibson (13678950),John Gibson,13678950,G,7700,VAN@ANH 11/01/2019 10:00PM ET,ANH,15.19,13.88,0.000000,0.000000
9,G,Carter Hart (13678951),Carter Hart,13678951,G,7600,PHI@NJ 11/01/2019 07:00PM ET,PHI,4.97,9.11,0.000000,0.000000
10,G,Thomas Greiss (13678965),Thomas Greiss,13678965,G,7600,TB@NYI 11/01/2019 07:00PM ET,NYI,16.90,16.68,11.984458,12.216659
13,G,Jacob Markstrom (13678953),Jacob Markstrom,13678953,G,7500,VAN@ANH 11/01/2019 10:00PM ET,VAN,15.42,13.46,13.498725,12.610475
